### TMS, Part 1
Goal: TMS, Part 1 after OOS completed

#### Pseudocode initialize and Out of Sample



1. Retrieve issue, system name, OOS trade data
2. Retrieve system dict
3. Set forecast parameters
4. Set start-end index
5. Range (start, end, interval)
	1. Initialize vars
	2. Set dd95
	3. While abs(dd95-ddTol) > 0.03 (repeat until dd95 is close to ddTolerance)
        1. Range (1,nCurves)
			1. Generate nCurves equity curves
			2. Store maxDD, TWR
			3. Compute dd95 (maxDD at tailRiskPct percentile)
			4. If dd95 ~= ddTolerance:
				1. Safe-f = fraction
				2. Exit
			5. Else
				1. Fraction = fraction * ddTolerance/dd95
				2. Continue
6. Save OOS trades data appended with safe-f, CAR25


##### Import standard libraries

In [31]:
# Import
%matplotlib inline

import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
import matplotlib.pylab as plt
from pandas.tseries.offsets import BDay
import os
import os.path
import pickle
import random
import json
from scipy import stats

from sklearn.model_selection import StratifiedShuffleSplit, TimeSeriesSplit
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

##### Import custom libraries

In [32]:
#
from Code.lib.plot_utils import PlotUtility
from Code.lib.time_utils import TimeUtility
from Code.lib.retrieve_data import DataRetrieve, ComputeTarget
from Code.lib.candle_indicators import CandleIndicators
from Code.lib.transformers import Transformers
from Code.lib.ta_momentum_studies import TALibMomentumStudies
from Code.lib.model_utils import ModelUtility, TimeSeriesSplitImproved
from Code.lib.feature_generator import FeatureGenerator
from Code.utilities.stat_tests import stationarity_tests
from Code.lib.config import current_feature, feature_dict
from Code.models import models_utils
from Code.lib.model_algos import AlgoUtility

plotIt = PlotUtility()
timeUtil = TimeUtility()
ct = ComputeTarget()
candle_ind = CandleIndicators()
dSet = DataRetrieve()
taLibMomSt = TALibMomentumStudies()
transf = Transformers()
modelUtil = ModelUtility()
featureGen = FeatureGenerator()
dSet = DataRetrieve()
modelAlgo = AlgoUtility()

In [33]:
# function to create system directory based on system name if one doesn't already exist
def get_system_dir(system_name):
    # Create system directory in current path
    current_directory = os.getcwd()
    system_directory = os.path.join(current_directory, system_name)
    if not os.path.exists(system_directory):
       os.makedirs(system_directory)
    return system_directory

In [34]:
# functions for save and load json files
def save_json(filename, json_file):
    # Save system_dict to file
    #filename = 'system_dict.json'
    file_path = os.path.join(system_directory, filename)
    with open(file_path, 'w') as fp:
        json.dump(json_file, fp, sort_keys=True, indent=4)
        
def load_json(file_path):
    with open(file_path, 'r') as fp:
        json_file = json.load(fp)
    return json_file

### Get system info

In [35]:
system_name = "TLT-Long-system-7045-V1"

In [36]:
# Get system info
current_directory = os.getcwd()
system_directory = os.path.join(current_directory, system_name)
if not os.path.exists(system_directory):
    print("system doesn't exist")
else:
    filename = 'system_dict.json'    
    file_path = os.path.join(system_directory, filename)
    system_dict = load_json(file_path)
    issue = system_dict["issue"]
    direction = system_dict["direction"]
    ver_num = system_dict["ver_num"]
    # Perhaps only load these when needed?
    pivotDate = system_dict["pivot_date"]
    is_oos_ratio = system_dict["is_oos_ratio"]
    oos_months = system_dict["oos_months"]
    segments = system_dict["segments"]

#### Retrieve data from csv

In [37]:
# get shadow trades
filename = "OOS_Equity_" + system_name + ".csv"
path = system_directory+ "\\" + filename
sst = pd.read_csv(path)
sst.tail(3)

,Date,signal,gainAhead,Close
82,2018-12-31,1,0.005267,121.51
83,2019-01-02,-1,0.011379,122.15
84,2019-01-03,-1,-0.011575,123.54


### Initialize dataframe

In [38]:
# nrows = sst.shape[0]

In [39]:
sst = sst.set_index(pd.DatetimeIndex(sst['Date']))
sst=sst.drop('Date', axis=1)
sst['safef'] = 0.0
sst['CAR25'] = 0.0
sst.head(2)

,signal,gainAhead,Close,safef,CAR25
Date,,,,,
2018-09-04,-1,-0.002666,120.03,0.0,0.0
2018-09-05,-1,0.003676,119.71,0.0,0.0


#### Get start and end date of data frame

In [40]:
print(sst.index[0])
print(sst.index[-1])

2018-09-04 00:00:00
2019-01-03 00:00:00


In [41]:
start = sst.index[0]
end = sst.index[-1]
updateInterval = 1

#### Initialize analysis variables
1. How do I get these figures dynamically?

In [42]:
f_days = sst.shape[0]
f_days

85

In [57]:
forecastHorizon = 84
initialEquity = 100000
ddTolerance = 0.10
tailRiskPct = 95
windowLength = 1*forecastHorizon
nCurves = 1000

years_in_forecast = forecastHorizon / 252.0

In [65]:
# Create tms_dict
tms_dict = {}
tms_dict = {'forecastHorizon' : forecastHorizon,
           'initialEquity'    : initialEquity,
           'ddTolerance'      : ddTolerance,
           'tailRiskPct'      : tailRiskPct,
           'windowLength'     : windowLength,
           'nCurves'          : nCurves,
           'updateInterval'   : 1
           }
save_json('tms_dict.json', tms_dict)

#### Work with index instead of dates

In [59]:
iStart = sst.index.get_loc(start)
print(iStart)
iEnd = sst.index.get_loc(end)
print(iEnd)

0
84


In [60]:
printDetails = False

In [61]:
# Calculate safe-f, CAR25
for i in range(iStart, iEnd+1, updateInterval):
    if printDetails: 
        print ("\nDate: ", dt.datetime.strftime(sst.index[i], '%Y-%m-%d'))
        print ("beLong: ", sst.signal[i])
        print ("gain Ahead: {0:.4f}".format(sst.gainAhead[i]))

#  Initialize variables
    curves = np.zeros(nCurves)
    numberDraws = np.zeros(nCurves)
    TWR = np.zeros(nCurves)
    maxDD = np.zeros(nCurves)
    
    fraction = 1.00
    dd95 = 2 * ddTolerance
    
    while (abs(dd95-ddTolerance)>0.03):
        #  Generate nCurve equity curves
        if printDetails: 
            print  ("    Fraction {0:.2f}".format(fraction))
#    
        for nc in range(nCurves):
            #print ("working on curve ", nc)
            equity = initialEquity
            maxEquity = equity
            drawdown = 0
            maxDrawdown = 0
            horizonSoFar = 0
            nd = 0
            while (horizonSoFar < forecastHorizon):
                j = np.random.randint(0,windowLength)
        #        print j
                nd = nd + 1
                weightJ = 1.00 - j/windowLength
        #        print weightJ
                horizonSoFar = horizonSoFar + weightJ
                signalJ = sst.signal[i-j]
                if signalJ > 0:
                    tradeJ = sst.gainAhead[i-j] * weightJ
                else:
                    tradeJ = 0.0
                thisTrade = fraction * tradeJ * equity    
                equity = equity + thisTrade
                maxEquity = max(equity,maxEquity)
                drawdown = (maxEquity-equity)/maxEquity
                maxDrawdown = max(drawdown,maxDrawdown)
    #        print "equity, maxDD, ndraws:", equity, maxDrawdown, nd        
            TWR[nc] = equity
            maxDD[nc] = maxDrawdown
            numberDraws[nc] = nd
    
        #  Find the drawdown at the tailLimit-th percentile        
        dd95 = stats.scoreatpercentile(maxDD,tailRiskPct)
        if printDetails: 
            print ('  DD {0}: {1:.3f} '.format(tailRiskPct, dd95))
        fraction = fraction * ddTolerance / dd95
        TWR25 = stats.scoreatpercentile(TWR,25)        
        CAR25 = 100*(((TWR25/initialEquity) ** (1.0/years_in_forecast))-1.0)
    if printDetails: 
        print ('Fraction: {0:.2f}'.format(fraction))
        print ('CAR25: {0:.2f}'.format(CAR25))
    sst.iloc[i,sst.columns.get_loc('safef')] = fraction
    sst.iloc[i,sst.columns.get_loc('CAR25')] = CAR25
    #sst.loc[i,'CAR25'] = CAR25

In [62]:
# print ("Max DD: {}".format(maxDD))        
# print ("Number of draws: {}".format(numberDraws))

In [63]:
sst.tail(3)

,signal,gainAhead,Close,safef,CAR25
Date,,,,,
2018-12-31,1,0.005267,121.51,4.207164,291.094845
2019-01-02,-1,0.011379,122.15,4.189522,275.851199
2019-01-03,-1,-0.011575,123.54,4.422409,288.965735


In [55]:
df_to_save = sst.copy()
df_to_save.reset_index(level=df_to_save.index.names, inplace=True)
filename = "TMS_Part1_" + system_name + ".csv"
df_to_save.to_csv(system_directory+ "\\" + filename, encoding='utf-8', index=False)

In [56]:
df_to_save.tail(3)

,Date,signal,gainAhead,Close,safef,CAR25
82,2018-12-31,1,0.005267,121.51,4.687286,310.685597
83,2019-01-02,-1,0.011379,122.15,4.659175,500.513987
84,2019-01-03,-1,-0.011575,123.54,4.785876,251.022485
